In [1]:
%load_ext autoreload
%autoreload 2

from util_0802_1 import *
from myalgorithm_0802_3 import algorithm
from myalgorithm_0802_5 import algorithm as algorithm_0802_5

import pandas as pd

## 실험 내용
- 크루스칼 돌릴 때 edge를 1/10 정도만 돌렸을 때 나오는 가중치 조합을 이용 가능한가? - 속도는 어떻게 차이가 나나?

### 데이터셋별 크루스칼 코드 속도 확인

In [2]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

results = []
for problem_file in problem_files:
    testcase_name = problem_file.split('\\')[-1].split('.')[0]

    start_time = time.time()

    ## ------------------- 기본 변수 할당 ---------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    timelimit = 60

    ## ------------------- 초기 상태 할당 코드 -------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    init_availables = [rider.available_number for rider in ALL_RIDERS]

    ## ------------------  크루스칼 함수   -----------------------------

    def kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, order_count_upper_limit, avg_method, all_bundles, default_get_dist_function):
        def find(v):
            while v != parent[v]:
                parent[v] = parent[parent[v]]
                v = parent[v]

            return v

        def union(a, b, new_bundle):
            if a > b:
                a, b = b, a

            parent[b] = a
            all_bundles[a] = new_bundle

        for i in range(len(all_bundles)):
            bundle = all_bundles[i]

            shop_seq = bundle.shop_seq

            xs_s_sum = 0
            ys_s_sum = 0

            xs_e_sum = 0
            ys_e_sum = 0

            readytimes_sum = 0
            deadlines_sum = 0

            shop_seq_len = len(shop_seq)

            for order_num in shop_seq:
                order = ALL_ORDERS[order_num]

                xs_s_sum += order.shop_lat
                ys_s_sum += order.shop_lon

                xs_e_sum += order.dlv_lat
                ys_e_sum += order.dlv_lon

                readytimes_sum += order.ready_time
                deadlines_sum += order.deadline

            xs_s_avg = xs_s_sum / shop_seq_len
            ys_s_avg = ys_s_sum / shop_seq_len

            xs_e_avg = xs_e_sum / shop_seq_len
            ys_e_avg = ys_e_sum / shop_seq_len

            readytimes_avg = readytimes_sum / shop_seq_len
            deadlines_avg = deadlines_sum / shop_seq_len

            avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

            bundle.avg_info = avg_info

        edges = []
        for i in range(len(all_bundles)):
            for j in range(i + 1, len(all_bundles)):
                avg_info1 = all_bundles[i].avg_info
                avg_info2 = all_bundles[j].avg_info

                sx1, sy1, ex1, ey1, r1, d1 = avg_info1
                sx2, sy2, ex2, ey2, r2, d2 = avg_info2

                r_diff = abs(r1 - r2)
                d_diff = abs(d1 - d2)

                start_end_diff = default_get_dist_function((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

                if avg_method == 'avg':
                    dist1 = default_get_dist_function(sx1, sy1, sx2, sy2)
                    dist2 = default_get_dist_function(ex1, ey1, ex2, ey2)
                elif avg_method == 'two_seq':
                    dist1 = DIST[i][j]
                    dist2 = DIST[i + K][j + K]
                elif avg_method == 'two':
                    order_num1 = all_bundles[i].shop_seq[0]
                    order_num2 = all_bundles[j].shop_seq[0]

                    dist1 = DIST[order_num1][order_num2]
                    dist2 = DIST[order_num1 + K][order_num2 + K]  
                else:
                    assert False

                # weight1 = (dist1 + dist2) / 900

                diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2
                # diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1
                # diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1

                edges.append((i, j, diff_score))

        parent = list(range(len(all_bundles)))
        edges.sort(key=lambda x: x[2])

        for bundle_num1, bundle_num2, diff_score in edges:
            rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

            if rbn1 == rbn2:
                continue

            new_bundle = bundle_merging_function(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2], order_count_upper_limit)

            if new_bundle is not None:
                all_bundles[rbn1].rider.available_number += 1
                all_bundles[rbn2].rider.available_number += 1
                
                new_bundle.rider.available_number -= 1

                union(rbn1, rbn2, new_bundle)

        parent = [find(v) for v in parent]

        result_bundles = [all_bundles[v] for v in set(parent)]
        rider_availables = [rider.available_number for rider in ALL_RIDERS]

        return result_bundles, rider_availables

    ## --------------- 초기 번들링 최적화 코드 --------------------------

    weight1 = 1
    weight2 = -2

    avg_method = 'two'
    bundle_merging_function = try_merging_bundles_by_dist
    default_get_dist_function = get_dist_by_coords

    inf = float('inf')

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    all_bundles = []
    for ord in ALL_ORDERS:
        new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
        car_rider.available_number -= 1
        all_bundles.append(new_bundle)

    # 2개 주문 묶음 생성
    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 2, 'two_seq', all_bundles, default_get_dist_function)

    # 4개 주문 묶음 생성
    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 4, 'avg', all_bundles, default_get_dist_function)

    to_check = [bundle for bundle in all_bundles if len(bundle.shop_seq) == 4]

    # 2개 이하 주문이 묶인 번들을 전부 푼 다음 다시 생성
    new_all_bundles = []
    for bundle in all_bundles:
        if len(bundle.shop_seq) >= 3:
            new_all_bundles.append(bundle)
        else:
            old_rider = bundle.rider
            old_rider.available_number += 1
            for order_num in bundle.shop_seq:
                order = ALL_ORDERS[order_num]

                new_bundle = Bundle(ALL_ORDERS, car_rider, [order.id], [order.id], order.volume, DIST[order.id, order.id + K])
                car_rider.available_number -= 1
                new_all_bundles.append(new_bundle)

    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 3, 'two', new_all_bundles, default_get_dist_function)

    ## ------------------- 라이더 재배치 -------------------------------

    all_bundles, rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
    for rider_i in range(3):
        ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

    ## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

    solution = [
            # rider type, shop_seq, dlv_seq
            [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
            for bundle in all_bundles
    ]

    # with open(problem_file, 'r') as f:
    #     prob = json.load(f)

    # K = prob['K']

    # ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    # ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    # DIST = np.array(prob['DIST'])
    # for r in ALL_RIDERS:
    #     r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    # checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    end_time = time.time()

    process_time = end_time - start_time

    results.append((process_time, K, testcase_name))

print(results)

[(0.5623748302459717, 50, 'TEST_K50_1'), (0.797548770904541, 50, 'TEST_K50_2'), (1.4590950012207031, 100, 'TEST_K100_1'), (2.8687286376953125, 100, 'TEST_K100_2'), (7.132972717285156, 200, 'TEST_K200_1'), (5.770768880844116, 200, 'TEST_K200_2'), (1.7393081188201904, 100, 'STAGE1_1'), (0.39453935623168945, 100, 'STAGE1_2'), (5.760993242263794, 200, 'STAGE1_3'), (1.0199401378631592, 200, 'STAGE1_4'), (4.142667531967163, 300, 'STAGE1_5'), (1.723175048828125, 300, 'STAGE1_6'), (1.6157300472259521, 100, 'STAGE1_7'), (0.47454261779785156, 100, 'STAGE1_8'), (8.043907165527344, 200, 'STAGE1_9'), (0.8271903991699219, 200, 'STAGE1_10'), (9.24054217338562, 300, 'STAGE1_11'), (2.0635297298431396, 300, 'STAGE1_12'), (2.830946207046509, 100, 'STAGE1_13'), (0.9337356090545654, 100, 'STAGE1_14'), (10.161705017089844, 200, 'STAGE1_15'), (1.0128638744354248, 200, 'STAGE1_16'), (20.307259559631348, 300, 'STAGE1_17'), (2.106745958328247, 300, 'STAGE1_18')]


In [9]:
results.sort(key=lambda x: (x[1], x[0]))

for tm, K, name in results:
    print(tm, K, name)

0.5623748302459717 50 TEST_K50_1
0.797548770904541 50 TEST_K50_2
0.39453935623168945 100 STAGE1_2
0.47454261779785156 100 STAGE1_8
0.9337356090545654 100 STAGE1_14
1.4590950012207031 100 TEST_K100_1
1.6157300472259521 100 STAGE1_7
1.7393081188201904 100 STAGE1_1
2.830946207046509 100 STAGE1_13
2.8687286376953125 100 TEST_K100_2
0.8271903991699219 200 STAGE1_10
1.0128638744354248 200 STAGE1_16
1.0199401378631592 200 STAGE1_4
5.760993242263794 200 STAGE1_3
5.770768880844116 200 TEST_K200_2
7.132972717285156 200 TEST_K200_1
8.043907165527344 200 STAGE1_9
10.161705017089844 200 STAGE1_15
1.723175048828125 300 STAGE1_6
2.0635297298431396 300 STAGE1_12
2.106745958328247 300 STAGE1_18
4.142667531967163 300 STAGE1_5
9.24054217338562 300 STAGE1_11
20.307259559631348 300 STAGE1_17


### 엣지 개수 1/10일시 데이터셋별 크루스칼 코드 속도 확인

In [7]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

results = []
for problem_file in problem_files:
    testcase_name = problem_file.split('\\')[-1].split('.')[0]

    start_time = time.time()

    ## ------------------- 기본 변수 할당 ---------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    timelimit = 60

    ## ------------------- 초기 상태 할당 코드 -------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    init_availables = [rider.available_number for rider in ALL_RIDERS]

    ## ------------------  크루스칼 함수   -----------------------------

    def kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, order_count_upper_limit, avg_method, all_bundles, default_get_dist_function):
        def find(v):
            while v != parent[v]:
                parent[v] = parent[parent[v]]
                v = parent[v]

            return v

        def union(a, b, new_bundle):
            if a > b:
                a, b = b, a

            parent[b] = a
            all_bundles[a] = new_bundle

        for i in range(len(all_bundles)):
            bundle = all_bundles[i]

            shop_seq = bundle.shop_seq

            xs_s_sum = 0
            ys_s_sum = 0

            xs_e_sum = 0
            ys_e_sum = 0

            readytimes_sum = 0
            deadlines_sum = 0

            shop_seq_len = len(shop_seq)

            for order_num in shop_seq:
                order = ALL_ORDERS[order_num]

                xs_s_sum += order.shop_lat
                ys_s_sum += order.shop_lon

                xs_e_sum += order.dlv_lat
                ys_e_sum += order.dlv_lon

                readytimes_sum += order.ready_time
                deadlines_sum += order.deadline

            xs_s_avg = xs_s_sum / shop_seq_len
            ys_s_avg = ys_s_sum / shop_seq_len

            xs_e_avg = xs_e_sum / shop_seq_len
            ys_e_avg = ys_e_sum / shop_seq_len

            readytimes_avg = readytimes_sum / shop_seq_len
            deadlines_avg = deadlines_sum / shop_seq_len

            avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

            bundle.avg_info = avg_info

        edges = []
        for i in range(len(all_bundles)):
            for j in range(i + 1, len(all_bundles)):
                avg_info1 = all_bundles[i].avg_info
                avg_info2 = all_bundles[j].avg_info

                sx1, sy1, ex1, ey1, r1, d1 = avg_info1
                sx2, sy2, ex2, ey2, r2, d2 = avg_info2

                r_diff = abs(r1 - r2)
                d_diff = abs(d1 - d2)

                start_end_diff = default_get_dist_function((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

                if avg_method == 'avg':
                    dist1 = default_get_dist_function(sx1, sy1, sx2, sy2)
                    dist2 = default_get_dist_function(ex1, ey1, ex2, ey2)
                elif avg_method == 'two_seq':
                    dist1 = DIST[i][j]
                    dist2 = DIST[i + K][j + K]
                elif avg_method == 'two':
                    order_num1 = all_bundles[i].shop_seq[0]
                    order_num2 = all_bundles[j].shop_seq[0]

                    dist1 = DIST[order_num1][order_num2]
                    dist2 = DIST[order_num1 + K][order_num2 + K]  
                else:
                    assert False

                # weight1 = (dist1 + dist2) / 900

                diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2
                # diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1
                # diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1

                edges.append((i, j, diff_score))

        parent = list(range(len(all_bundles)))
        edges.sort(key=lambda x: x[2])
        edges = edges[:int(len(edges) // 10)]

        for bundle_num1, bundle_num2, diff_score in edges:
            rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

            if rbn1 == rbn2:
                continue

            new_bundle = bundle_merging_function(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2], order_count_upper_limit)

            if new_bundle is not None:
                all_bundles[rbn1].rider.available_number += 1
                all_bundles[rbn2].rider.available_number += 1
                
                new_bundle.rider.available_number -= 1

                union(rbn1, rbn2, new_bundle)

        parent = [find(v) for v in parent]

        result_bundles = [all_bundles[v] for v in set(parent)]
        rider_availables = [rider.available_number for rider in ALL_RIDERS]

        return result_bundles, rider_availables

    ## --------------- 초기 번들링 최적화 코드 --------------------------

    weight1 = 1
    weight2 = -2

    avg_method = 'two'
    bundle_merging_function = try_merging_bundles_by_dist
    default_get_dist_function = get_dist_by_coords

    inf = float('inf')

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    all_bundles = []
    for ord in ALL_ORDERS:
        new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
        car_rider.available_number -= 1
        all_bundles.append(new_bundle)

    # 2개 주문 묶음 생성
    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 2, 'two_seq', all_bundles, default_get_dist_function)

    # 4개 주문 묶음 생성
    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 4, 'avg', all_bundles, default_get_dist_function)

    to_check = [bundle for bundle in all_bundles if len(bundle.shop_seq) == 4]

    # 2개 이하 주문이 묶인 번들을 전부 푼 다음 다시 생성
    new_all_bundles = []
    for bundle in all_bundles:
        if len(bundle.shop_seq) >= 3:
            new_all_bundles.append(bundle)
        else:
            old_rider = bundle.rider
            old_rider.available_number += 1
            for order_num in bundle.shop_seq:
                order = ALL_ORDERS[order_num]

                new_bundle = Bundle(ALL_ORDERS, car_rider, [order.id], [order.id], order.volume, DIST[order.id, order.id + K])
                car_rider.available_number -= 1
                new_all_bundles.append(new_bundle)

    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 3, 'two', new_all_bundles, default_get_dist_function)

    ## ------------------- 라이더 재배치 -------------------------------

    all_bundles, rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
    for rider_i in range(3):
        ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

    ## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

    solution = [
            # rider type, shop_seq, dlv_seq
            [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
            for bundle in all_bundles
    ]

    # with open(problem_file, 'r') as f:
    #     prob = json.load(f)

    # K = prob['K']

    # ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    # ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    # DIST = np.array(prob['DIST'])
    # for r in ALL_RIDERS:
    #     r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    # checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    end_time = time.time()

    process_time = end_time - start_time

    avg_cost = sum((bundle.cost for bundle in all_bundles)) / K

    results.append((process_time, K, testcase_name, avg_cost))

print(results)

[(0.0634908676147461, 50, 'TEST_K50_1', 3784.8320000000003), (0.07114744186401367, 50, 'TEST_K50_2', 4288.829999999999), (0.22352957725524902, 100, 'TEST_K100_1', 3635.9800000000005), (0.29150843620300293, 100, 'TEST_K100_2', 3881.9840000000004), (0.9277784824371338, 200, 'TEST_K200_1', 3477.8515), (0.7591509819030762, 200, 'TEST_K200_2', 3430.1275000000005), (0.20646286010742188, 100, 'STAGE1_1', 3930.2140000000004), (0.14162778854370117, 100, 'STAGE1_2', 3615.4760000000015), (0.6139891147613525, 200, 'STAGE1_3', 3173.3374999999996), (0.35700130462646484, 200, 'STAGE1_4', 2841.258), (0.941596269607544, 300, 'STAGE1_5', 2332.8340000000007), (0.671910285949707, 300, 'STAGE1_6', 2694.095666666666), (0.18021535873413086, 100, 'STAGE1_7', 3623.14), (0.12372994422912598, 100, 'STAGE1_8', 3509.492), (0.7163894176483154, 200, 'STAGE1_9', 3244.5164999999997), (0.3176124095916748, 200, 'STAGE1_10', 2708.6859999999997), (1.158341884613037, 300, 'STAGE1_11', 2645.569), (0.5606136322021484, 300, '

In [9]:
results.sort(key=lambda x: (x[1], x[0]))

for tm, K, name, avg_cost in results:
    print(tm, K, name, avg_cost)

0.0634908676147461 50 TEST_K50_1 3784.8320000000003
0.07114744186401367 50 TEST_K50_2 4288.829999999999
0.12372994422912598 100 STAGE1_8 3509.492
0.14162778854370117 100 STAGE1_2 3615.4760000000015
0.1510305404663086 100 STAGE1_14 4486.502
0.18021535873413086 100 STAGE1_7 3623.14
0.20646286010742188 100 STAGE1_1 3930.2140000000004
0.22352957725524902 100 TEST_K100_1 3635.9800000000005
0.2892954349517822 100 STAGE1_13 5039.494
0.29150843620300293 100 TEST_K100_2 3881.9840000000004
0.3176124095916748 200 STAGE1_10 2708.6859999999997
0.35700130462646484 200 STAGE1_4 2841.258
0.4827723503112793 200 STAGE1_16 3430.4350000000004
0.6139891147613525 200 STAGE1_3 3173.3374999999996
0.7163894176483154 200 STAGE1_9 3244.5164999999997
0.7591509819030762 200 TEST_K200_2 3430.1275000000005
0.9277784824371338 200 TEST_K200_1 3477.8515
1.1129086017608643 200 STAGE1_15 4412.346
0.5606136322021484 300 STAGE1_12 2601.5476666666664
0.6658449172973633 300 STAGE1_18 3734.645
0.671910285949707 300 STAGE1_6 2

### 엣지 수 1/10을 활용한 최적 가중치 탐색

In [ ]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

results = []
for problem_file in problem_files:
    testcase_name = problem_file.split('\\')[-1].split('.')[0]

    start_time = time.time()

    ## ------------------- 기본 변수 할당 ---------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    timelimit = 60

    ## ------------------- 초기 상태 할당 코드 -------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    init_availables = [rider.available_number for rider in ALL_RIDERS]

    ## ------------------  크루스칼 함수   -----------------------------

    def kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, order_count_upper_limit, avg_method, all_bundles, default_get_dist_function):
        def find(v):
            while v != parent[v]:
                parent[v] = parent[parent[v]]
                v = parent[v]

            return v

        def union(a, b, new_bundle):
            if a > b:
                a, b = b, a

            parent[b] = a
            all_bundles[a] = new_bundle

        for i in range(len(all_bundles)):
            bundle = all_bundles[i]

            shop_seq = bundle.shop_seq

            xs_s_sum = 0
            ys_s_sum = 0

            xs_e_sum = 0
            ys_e_sum = 0

            readytimes_sum = 0
            deadlines_sum = 0

            shop_seq_len = len(shop_seq)

            for order_num in shop_seq:
                order = ALL_ORDERS[order_num]

                xs_s_sum += order.shop_lat
                ys_s_sum += order.shop_lon

                xs_e_sum += order.dlv_lat
                ys_e_sum += order.dlv_lon

                readytimes_sum += order.ready_time
                deadlines_sum += order.deadline

            xs_s_avg = xs_s_sum / shop_seq_len
            ys_s_avg = ys_s_sum / shop_seq_len

            xs_e_avg = xs_e_sum / shop_seq_len
            ys_e_avg = ys_e_sum / shop_seq_len

            readytimes_avg = readytimes_sum / shop_seq_len
            deadlines_avg = deadlines_sum / shop_seq_len

            avg_info = [xs_s_avg, ys_s_avg, xs_e_avg, ys_e_avg, readytimes_avg, deadlines_avg]

            bundle.avg_info = avg_info

        edges = []
        for i in range(len(all_bundles)):
            for j in range(i + 1, len(all_bundles)):
                avg_info1 = all_bundles[i].avg_info
                avg_info2 = all_bundles[j].avg_info

                sx1, sy1, ex1, ey1, r1, d1 = avg_info1
                sx2, sy2, ex2, ey2, r2, d2 = avg_info2

                r_diff = abs(r1 - r2)
                d_diff = abs(d1 - d2)

                start_end_diff = default_get_dist_function((sx1 + sx2) / 2, (sy1 + sy2) / 2, (ex1 + ex2) / 2, (ey1 + ey2) / 2)

                if avg_method == 'avg':
                    dist1 = default_get_dist_function(sx1, sy1, sx2, sy2)
                    dist2 = default_get_dist_function(ex1, ey1, ex2, ey2)
                elif avg_method == 'two_seq':
                    dist1 = DIST[i][j]
                    dist2 = DIST[i + K][j + K]
                elif avg_method == 'two':
                    order_num1 = all_bundles[i].shop_seq[0]
                    order_num2 = all_bundles[j].shop_seq[0]

                    dist1 = DIST[order_num1][order_num2]
                    dist2 = DIST[order_num1 + K][order_num2 + K]  
                else:
                    assert False

                # weight1 = (dist1 + dist2) / 900

                diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1 + start_end_diff * weight2
                # diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1
                # diff_score = dist1 + dist2 + r_diff * weight1 + d_diff * weight1

                edges.append((i, j, diff_score))

        parent = list(range(len(all_bundles)))
        edges.sort(key=lambda x: x[2])
        edges = edges[:int(len(edges) // 10)]

        for bundle_num1, bundle_num2, diff_score in edges:
            rbn1, rbn2 = find(bundle_num1), find(bundle_num2)

            if rbn1 == rbn2:
                continue

            new_bundle = bundle_merging_function(K, DIST, ALL_ORDERS, ALL_RIDERS, all_bundles[rbn1], all_bundles[rbn2], order_count_upper_limit)

            if new_bundle is not None:
                all_bundles[rbn1].rider.available_number += 1
                all_bundles[rbn2].rider.available_number += 1
                
                new_bundle.rider.available_number -= 1

                union(rbn1, rbn2, new_bundle)

        parent = [find(v) for v in parent]

        result_bundles = [all_bundles[v] for v in set(parent)]
        rider_availables = [rider.available_number for rider in ALL_RIDERS]

        return result_bundles, rider_availables

    ## --------------- 초기 번들링 최적화 코드 --------------------------

    weight1 = 1
    weight2 = -2

    avg_method = 'two'
    bundle_merging_function = try_merging_bundles_by_dist
    default_get_dist_function = get_dist_by_coords

    inf = float('inf')

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    all_bundles = []
    for ord in ALL_ORDERS:
        new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
        car_rider.available_number -= 1
        all_bundles.append(new_bundle)

    # 2개 주문 묶음 생성
    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 2, 'two_seq', all_bundles, default_get_dist_function)

    # 4개 주문 묶음 생성
    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 4, 'avg', all_bundles, default_get_dist_function)

    to_check = [bundle for bundle in all_bundles if len(bundle.shop_seq) == 4]

    # 2개 이하 주문이 묶인 번들을 전부 푼 다음 다시 생성
    new_all_bundles = []
    for bundle in all_bundles:
        if len(bundle.shop_seq) >= 3:
            new_all_bundles.append(bundle)
        else:
            old_rider = bundle.rider
            old_rider.available_number += 1
            for order_num in bundle.shop_seq:
                order = ALL_ORDERS[order_num]

                new_bundle = Bundle(ALL_ORDERS, car_rider, [order.id], [order.id], order.volume, DIST[order.id, order.id + K])
                car_rider.available_number -= 1
                new_all_bundles.append(new_bundle)

    all_bundles, rider_availables = kruskal_bundling(K, DIST, ALL_ORDERS, ALL_RIDERS, weight1, weight2, bundle_merging_function, 3, 'two', new_all_bundles, default_get_dist_function)

    ## ------------------- 라이더 재배치 -------------------------------

    all_bundles, rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
    for rider_i in range(3):
        ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

    ## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

    solution = [
            # rider type, shop_seq, dlv_seq
            [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
            for bundle in all_bundles
    ]

    # with open(problem_file, 'r') as f:
    #     prob = json.load(f)

    # K = prob['K']

    # ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    # ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    # DIST = np.array(prob['DIST'])
    # for r in ALL_RIDERS:
    #     r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    # checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    end_time = time.time()

    process_time = end_time - start_time

    avg_cost = sum((bundle.cost for bundle in all_bundles)) / K

    results.append((process_time, K, testcase_name, avg_cost))

print(results)

### 모든 데이터셋에 걸친 엣지 수 1/10을 활용한 최적 가중치 탐색

In [ ]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

results = []
for problem_file in problem_files:
    ## ------------------- 기본 변수 할당 ---------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    timelimit = 60

    ## --------------------- 커스텀 코드 시작 ---------------------------

    solution = algorithm(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    print(problem_file)

    results.append((problem_file, checked_solution['avg_cost']))

df = pd.DataFrame(results, columns=['Name', 'Score'])

df.to_csv('8-1_siwoo_changed_20sec_edge_num_divided_by_10.csv', index=False, encoding='utf-8-sig')

### 모든 데이터셋에 걸친 엣지 수 1/10을 활용한 가중치 탐색 우선순위 부여

In [3]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

results = []
for problem_file in problem_files:
    ## ------------------- 기본 변수 할당 ---------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    timelimit = 60

    ## --------------------- 커스텀 코드 시작 ---------------------------

    solution = algorithm_0802_5(K, ALL_ORDERS, ALL_RIDERS, DIST, timelimit)

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

    print(problem_file)

    results.append((problem_file, checked_solution['avg_cost']))

df = pd.DataFrame(results, columns=['Name', 'Score'])

df.to_csv('8-1_siwoo_changed_10to30sec_edge_num_divided_by_10.csv', index=False, encoding='utf-8-sig')

C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K50_1.json
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K50_2.json
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K100_1.json
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K100_2.json
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K200_1.json
C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K200_2.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_1.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_2.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_3.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_4.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_5.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_6.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_7.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_8.json
C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_9.json
C:\Users\hsh80\Des